In [1]:
import os

In [2]:
%pwd

'c:\\Users\\kk061\\OneDrive\\Desktop\\python\\Mlflow Main\\Heart Stroke Prediction\\notebook'

In [3]:
os.chdir('../')
%pwd

'c:\\Users\\kk061\\OneDrive\\Desktop\\python\\Mlflow Main\\Heart Stroke Prediction'

In [4]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/kk1455217/Heart-Stroke-Prediction.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="kk1455217"
os.environ["MLFLOW_TRACKING_PASSWORD"]="9c05d7cb76433d44cb358f7431105d416ba10413"

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    pkl_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [6]:
from src.heartstrokeprediction.constants import *
from src.heartstrokeprediction.utils.common import read_yaml, create_directories,save_json

[2025-04-09 17:21:48,402: INFO: __init__: This is an info message.]
[2025-04-09 17:21:48,404: ERROR: __init__: This is an error message.]


In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.LogisticRegression
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            pkl_path=config.pkl_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/kk1455217/Heart-Stroke-Prediction.mlflow"
        )
        return model_evaluation_config

In [8]:
import os
from src.heartstrokeprediction import logger
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
import seaborn as sns
import matplotlib.pyplot as plt

In [9]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        """Calculate evaluation metrics: RMSE, MAE, R2"""
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def log_into_mlflow(self):
        """Evaluate model and log results into MLflow"""
        # Load the test data
        test_data = pd.read_csv(self.config.test_data_path)

        # Load the model and scaler
        model = joblib.load(self.config.model_path)
        scaler = joblib.load(self.config.pkl_path)

        # Separate features (X) and target (y)
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        # Scale the test features using the saved scaler
        test_x_scaled = scaler.transform(test_x)

        # Start an MLflow run
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            # Make predictions on the scaled test data
            predicted_qualities = model.predict(test_x_scaled)

            # Evaluate the model and calculate metrics
            rmse, mae, r2 = self.eval_metrics(test_y, predicted_qualities)

            # Save the metrics as a JSON file
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            # Log parameters to MLflow
            mlflow.log_params(self.config.all_params)

            # Log metrics to MLflow
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            # If not using a file store, register the model in the Model Registry
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="HeartStrokePredictionModel")
            else:
                mlflow.sklearn.log_model(model, "model")

            print(f"Evaluation metrics saved and model logged in MLflow!")

In [10]:
try:
    # Load configuration for evaluation
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()

    # Instantiate ModelEvaluator and evaluate
    evaluator = ModelEvaluation(config=model_evaluation_config)
    evaluator.log_into_mlflow()

except Exception as e:
    logger.error(f"Error in model evaluation: {str(e)}")
    raise e

[2025-04-09 17:22:00,681: INFO: common: YAML file: config\config.yaml loaded successfully]
[2025-04-09 17:22:00,685: INFO: common: YAML file: params.yaml loaded successfully]
[2025-04-09 17:22:00,690: INFO: common: YAML file: schema.yaml loaded successfully]
[2025-04-09 17:22:00,692: INFO: common: created directory at: artifacts]
[2025-04-09 17:22:00,695: INFO: common: created directory at: artifacts/model_evaluation]
[2025-04-09 17:22:01,793: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


2025/04/09 17:22:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'HeartStrokePredictionModel' already exists. Creating a new version of this model...
2025/04/09 17:22:20 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: HeartStrokePredictionModel, version 3
Created version '3' of model 'HeartStrokePredictionModel'.


Evaluation metrics saved and model logged in MLflow!
🏃 View run auspicious-eel-374 at: https://dagshub.com/kk1455217/Heart-Stroke-Prediction.mlflow/#/experiments/0/runs/1cfa307fd04540bf8ede3af5c5be5d69
🧪 View experiment at: https://dagshub.com/kk1455217/Heart-Stroke-Prediction.mlflow/#/experiments/0
